In [ ]:
import urllib, requests,regex
import csv #CSVを扱えるようにするため
import re #文字列処理をしやすくするため
import time #処理を一時的に止めるため
import socks,socket #IPを偽装するため
import sys #ファイルのパスの情報を扱うため
from bs4 import BeautifulSoup #タグなどの抽出のため
from urllib.request import urlopen #情報取得のため
from requests import ConnectionError,Timeout #エラーが起きた際に回避するため
from urllib.error import HTTPError, URLError #エラーが起きた際に回避するため
from ssl import SSLError #エラーが起きた際に回避するため
import http.client #情報取得のため
import chardet #文字列を扱いやすくするライブラリ
from socks import SOCKS5Error #!pip install PySocks #エラーが起きた際に回避するため
socks.setdefaultproxy(socks.PROXY_TYPE_SOCKS5, "127.0.0.1", 9150) #IPを偽装するため
socket.socket = socks.socksocket #IPを偽装するため
sys.setrecursionlimit(10000) #処理量の多くなるようなサイトの時に取得が途中で止まらないようにするため

In [11]:
import pandas as pd
result_database = pd.DataFrame(columns=["url","company_name","company_url","job","address","establish","capital","contact"])

In [ ]:
url_list_1 = pd.read_csv("url_company_se_list.csv",header=None)
url_list_2 = pd.read_csv("url_company_soft_list.csv",header=None)
url_list_3 = pd.read_csv("url_company_creative_list.csv",header=None)

In [12]:
#　リスト3つを繋がる
url_list_all = url_list_1.append(url_list_2, ignore_index=True)
url_list_all = url_list_all.append(url_list_3, ignore_index=True)
url_list_all.columns = ["url"]

In [ ]:
# すでに取得したものを除外する
extracted_result = pd.read_pickle("result_database.pickle")
url_list_all = url_list_all[~url_list_all.url.isin(extracted_result.url)]

In [ ]:


for url in url_list_all[0]:


    company_url_sub=BeautifulSoup('None', "html.parser")
    company_name_sub=BeautifulSoup('None', "html.parser")
    contact_sub=BeautifulSoup('None', "html.parser")
    job_sub=BeautifulSoup('None', "html.parser")
    address_sub=BeautifulSoup('None', "html.parser")
    establish_sub=BeautifulSoup('None', "html.parser")
    capital_sub=BeautifulSoup('None', "html.parser")

    
    try:
        r = requests.get(url,timeout=60,allow_redirects=False)                   
        soup = BeautifulSoup(r.content, 'html.parser')

        remove_expression = "\r|\n|\t"

        for company_name in soup.findAll('h1',{'class':'iconDisp'}):
            company_name_sub = company_name.text            
            company_name_sub = re.sub(remove_expression,"",company_name_sub)

        for table in soup.findAll('div',{'class':'modDetail02'}):
            for company_url in table.findAll('dl'):
                for parts in company_url.findAll('dt'):
                    if regex.findall("企業URL",parts.text):
                        company_url_sub = company_url.find('dd')
                        company_url_sub = company_url_sub.find('a')
                        company_url_sub = company_url_sub.get('href')    


        for job in soup.findAll('p',{'class':'fs15 bold'}):
            job_sub = job.text
            job_sub =re.sub(remove_expression,"",job_sub) #イケてない文字列を削除する


        for table in soup.findAll('div',{'class':'modDetail04'}):
            for address in table.findAll('dl'):
                for parts in address.find('dt'):
                    if regex.findall("所在地",parts):
                        address_sub = address.find('dd')
                        address_sub = address_sub.text
                        address_sub =re.sub(remove_expression,"",address_sub) #イケてない文字列を削除する


        for table in soup.findAll('div',{'class':'modDetail04'}):
            for establish in table.findAll('dl'):
                for parts in establish.find('dt'):
                    if regex.findall("設立",parts):
                        establish_sub = establish.find('dd')
                        establish_sub = establish_sub.text
                        establish_sub =re.sub(remove_expression,"",establish_sub) #イケてない文字列を削除する

        for table in soup.findAll('div',{'class':'modDetail04'}):
            for capital in table.findAll('dl'):
                for parts in capital.find('dt'):
                    if regex.findall("資本金",parts):
                        capital = capital.find('dd')
                        capital_sub =capital.text
                        capital_sub =re.sub(remove_expression,"",capital_sub) #イケてない文字列を削除する

        for table in soup.findAll('div',{'class':'modDetail02'}):
            for contact in table.findAll('dl'):
                if regex.findall("連絡先",contact.text):
                    contact_sub = contact.text
                    contact_sub =re.sub(remove_expression,"",contact_sub) #イケてない文字列を削除する

        df = pd.DataFrame([[url,company_name_sub,company_url_sub,job_sub,address_sub,establish_sub,capital_sub,contact_sub]],columns=result_database.columns)            
        result_database = result_database.append(df).reset_index(drop=True) #インデックスをつける。付けないと何番目か分からない。
        result_database.to_pickle("result_database.pickle")
    
    

    except HTTPError as e:
        print(e.code)

    except requests.exceptions.Timeout:
        print("Timeout occurred")

    except ConnectionError:
        print("ConnectionError")

    except SSLError:
        print("SSLError")

    except TypeError:
        print("TypeError")

    except URLError:
        print("URLError")

    except  ValueError:
        print("ValueError")

    except socket.timeout:
        print("Timeout occurred")

    except KeyError as instance:
        print(instance)

    except ConnectionResetError:
        print("ConnectionResetError")

    except http.client.IncompleteRead:
        print("IncompleteRead")

    except http.client.HTTPException:
        print("HTTPException")

    except NotImplementedError:
        print("NotImplementedError")
